In [1]:
# importing libraries
from datetime import datetime
import mtcnn as Original_MTCNN
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
from torchvision import transforms
import glob
%matplotlib inline
import pandas as pd
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
omtcnn = Original_MTCNN.MTCNN()

In [3]:
# omtcnn.detect_faces

In [4]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [5]:
mtcnn = MTCNN(device = device)
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

dataset=datasets.ImageFolder('Images/train/')
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} 
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] 
name_list = [] 
embedding_list = [] 


In [6]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='face_detection_yolov5s.pt',device='cuda:0')

for imgs, idx in loader:

    transform1 = transforms.ToTensor()
    tensor = transform1(imgs)
    t = transforms.functional.adjust_gamma(tensor, gamma=2, gain=3)

    transform2 = transforms.ToPILImage()


    imgss = transform2(t)

    img =  np.array(imgss)

    results = model(img)
    
    
    
    for faces in results.xyxy[0]:
        if faces is None: continue
        else:
            faces = np.array(faces.detach().cpu())

            x = round(faces[0])
            y = round(faces[1])
            w = round(faces[2])
            h = round(faces[3])

            img_ = img[y-15:h+35, x-15:w+15]

            try: face = mtcnn(img_)
            except NotImplementedError: continue

            if face is not None:
                emb = resnet(face.unsqueeze(0).cuda()) 
                embedding_list.append(emb.detach()) 
                name_list.append(idx_to_class[idx])
            else: continue

Using cache found in C:\Users\a.nasimi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-18 Python-3.9.12 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)

Fusing layers... 
newYOLOv5s summary: 224 layers, 7053910 parameters, 0 gradients
Adding AutoShape... 
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: 

C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\A

C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\AI\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
C:\Users\a.nasimi\Anaconda3\envs\A

In [7]:
from threading import Thread

class ThreadedCamera(object):
    def __init__(self, source='cam:Sayda@0.0.0.0:88/videoMain'):

        self.capture = cv2.VideoCapture(f'rtsp://{source}')

        self.thread = Thread(target = self.update, args = ())
        self.thread.daemon = True
        self.thread.start()

        self.status = False
        self.frame  = None

    def update(self):
        while True:
            if self.capture.isOpened(): (self.status, self.frame) = self.capture.read()
            else: continue

    def grab_frame(self):
        if self.status: return self.frame
        else: return None 
    
    def end_frame(self):
        self.capture.release()

In [8]:
def Recognition(img,thresh=0.9):
    
    try: results = model(img)
    except AttributeError: return None
    
    if len(results.xyxy[0]) == 0: pass
    else:
        for res in results.xyxy[0]:
            
            res = np.array(res.detach().cpu())
            
            x = round(res[0])
            y = round(res[1])
            w = round(res[2])
            h = round(res[3])
            
            img_ = img[y-15:h+35, x-15:w+15]

            try: new = Image.fromarray(img_)
            except ValueError: new = Image.fromarray(img[y:h, x:w])

            face = mtcnn(new)
            if face is None: continue
            else:
                emb = resnet(face.unsqueeze(0).cuda()).detach() 

                saved_data = torch.load('data.pt') 
                embedding_list = saved_data[0] 
                name_list = saved_data[1] 
                dist_list,person = [],'Unknown' 

                for indx, emb_db in enumerate(embedding_list):
                    dist = torch.cdist(emb, emb_db, p=2).item()
                    dist_list.append(dist)
                    print(dist)
                    if dist < thresh: person = name_list[indx]
                    else: pass
#                     print(dist)

#                 idx_min = dist_list.index(min(dist_list))

                img = cv2.rectangle(np.array(img), (x,y,w-x,h-y), (0,0,255), 2)
                img = cv2.putText(img, text = person, org=(x, y), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 255), thickness=1)
    
    return img

In [9]:
frame_list,counter = [],0
streamer = ThreadedCamera('rtsp:Sayda@0.0.0.0')

In [ ]:
while True: 
    img = streamer.grab_frame()
    
    try: results = model(img)
    except AttributeError: continue
    
    if len(results.xyxy[0]) == 0: pass
    else:
        for indx,res in enumerate(results.xyxy[0]):
            res = np.array(res.detach().cpu())
            
            x = round(res[0])
            y = round(res[1])
            w = round(res[2])
            h = round(res[3])
            predict = results.pandas().xyxy[0]['name'][indx]
            
            img = cv2.rectangle(np.array(img), (x,y,w-x,h-y), (0,0,255), 2)
            img = cv2.putText(img, text = predict,qq org=(x, y), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 255), thickness=1)
    
    
    now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
    cv2.imshow('Emotion Detection', np.array(img))

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        cv2.destroyAllWindows()
        streamer.end_frame()
        break

    
    
#     if img is None: continue
#     else:
#         img_res = Recognition(img,1)

#         if img_res is None: cv2.imshow('Emotion Detection', np.array(img))
#         else:
#             now_time = datetime.now().strftime("%Y_%m_%d---%I_%M_%S---%p")
#             cv2.imshow('Emotion Detection', np.array(img_res))
#             cv2.imwrite(f'Log/{now_time}.jpg', img_res)
        
#         if cv2.waitKey(1) & 0xFF == ord('q'): 
#             cv2.destroyAllWindows()
#             streamer.end_frame()
#             break